In [ ]:
!pip install transformers datasets sentencepiece rouge-score openpyxl
!pip install torch torchvision torchaudio

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4bcc82b86b2ecf447f3dc99c94e398d90bdcebf369070c6b00a93857c4c4196a
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━

In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import BartForConditionalGeneration, BartTokenizer, Trainer, TrainingArguments
from datasets import Dataset
from rouge_score import rouge_scorer

In [ ]:
dataset_path = "/content/drive/MyDrive/The_Hindu_paper_dataset_2015_to_2025.xlsx"
df = pd.read_excel(dataset_path)
df.head()

,Date Published,Headline,URL,Content
0,2015-01-25,"Saina beats World Champ, retains Syed Modi crown",https://www.thehindu.com/sport/other-sports/sa...,"July 15, 2025e-Paper The View From IndiaLookin..."
1,2015-01-05,Indian weight lifters end campaign with 14 medals,https://www.thehindu.com/sport/other-sports/in...,"July 15, 2025e-Paper The View From IndiaLookin..."
2,2015-01-27,On his own track,https://www.thehindu.com/sport/other-sports/ra...,"July 15, 2025e-Paper The View From IndiaLookin..."
3,2015-01-06,"Now, Vijender Singh wants to try his luck",https://www.thehindu.com/sport/other-sports/af...,"July 7, 2025e-Paper The View From IndiaLooking..."
4,2015-01-12,Mary Kom mulls retirement after 2016 Olympics,https://www.thehindu.com/sport/other-sports/ma...,"July 15, 2025e-Paper The View From IndiaLookin..."


In [ ]:
df = df.fillna('')
df = df.drop_duplicates()
df['Content'] = df['Content'].astype(str)
df['Headline'] = df['Headline'].astype(str)
df = df.reset_index(drop = True)
print("Preprocessing complete. Dataframe info after preprocessing:")
df.info()

Preprocessing complete. Dataframe info after preprocessing:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1221 entries, 0 to 1220
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Date Published  1221 non-null   object
 1   Headline        1221 non-null   object
 2   URL             1221 non-null   object
 3   Content         1221 non-null   object
dtypes: object(4)
memory usage: 38.3+ KB


In [ ]:
def split_dataset(df, train_size = 0.8, val_size = 0.1, test_size = 0.1):
  if(train_size + val_size + test_size != 1.0):
    raise ValueError("The sum of train_size, val_size, and test_size must be 1.0")
  train_df , temp_df = train_test_split(df, test_size = (val_size + test_size), random_state = 42)
  adjusted_test_size = test_size/(test_size+val_size)
  val_df, test_df = train_test_split(df, test_size = adjusted_test_size, random_state = 42)
  return train_df, val_df, test_df

train_df, test_df, val_df = split_dataset(df)
print("Training set size:", len(train_df))
print("Validation set size:", len(val_df))
print("Testing set size:", len(test_df))

Training set size: 976
Validation set size: 611
Testing set size: 610


In [ ]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
max_input_length = 512
max_target_length = 128

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
def tokenize_data(tokenizer, dataframe, max_length):
    inputs = tokenizer(
        dataframe["Content"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )
    labels = tokenizer(
        dataframe["Headline"],
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt",
    )
    inputs['labels'] = labels['input_ids']
    return inputs

train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(lambda x: tokenize_data(tokenizer, x, max_input_length), batched=True)
val_dataset = val_dataset.map(lambda x: tokenize_data(tokenizer, x, max_input_length), batched=True)

print(train_dataset)

Map:   0%|          | 0/976 [00:00<?, ? examples/s]

Map:   0%|          | 0/611 [00:00<?, ? examples/s]

Dataset({
    features: ['Date Published', 'Headline', 'URL', 'Content', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 976
})


In [ ]:
training_args = TrainingArguments(
    output_dir = './bart_hindu_results',
    eval_strategy = 'epoch',
    save_strategy = 'epoch',
    per_device_train_batch_size = 2,
    per_device_eval_batch_size = 2,
    learning_rate = 5e-5,
    num_train_epochs = 10,
    weight_decay = 0.01,
    logging_steps = 50,
    logging_dir = './bart_hindu_logs',
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

/tmp/ipython-input-14-2851889554.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: rishipramodc (rishipramodc-indian-institute-of-information-technology-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
